In [1]:
# Class to define entries in the HashTable
class TableEntry:

    def __init__(self, key, value):
        self._key = key
        self._value = value
        self._in_table = True
        
    def get_key(self):
        return self._key
    
    def get_value(self):
        return self._value
    
    def set_value(self, value):
        self._value = value
    
    def is_in_table(self):
        return self._in_table
    
    def set_to_removed(self):
        self._key = None
        self._value = None
        self._in_table = False
    
    def is_removed(self):
        return (self._key is None) and (self._value is None) and not self._in_table

In [2]:
# Hash Table implementation
class HashTableWithCount:
    _LOAD_FACTOR_FOR_RESIZING = 0.7 # expand when 70% full
    
    def __init__(self, initial_capacity=997): # choose an initial capacity
        # array of buckets, initially all empty
        self._arr = [None] * initial_capacity
        self._capacity = initial_capacity
        self._size = 0
        self._probe_count = 0 # number of probes (steps) taken in last operation
    
    
    def get_next_prime(self, n):
        # find the next prime number greater than n
        # used to choose good table sizes for hashing
        # Time Complexity: O( sqrt(n) ) for primality checking
        n += 1
        while not self.is_prime(n):
            # always move to the next odd number
            if n % 2 == 0:
                n += 1
            else:
                n += 2
        return n

    def is_prime(self, n):
        # check if n is prime by testing divisors up to sqrt(n)
        # Time Complexity: O( sqrt(n) )
        if n <= 1: # less or equal than 1 is not prime
            return False
        if n == 2: # the only even prime number
            return True
        if n % 2 == 0: # all other even number are not prime
            return False
        
        # try odd divisors from 3 to sqrt(n)
        for i in range(3, int(n ** 0.5) + 1, 2):
            if n % i == 0:
                return False
        # If no divisors are found, the number is prime
        return True
    
    def check_size(self, size):
        # compute current load factor (occupied slots / table size)
        # Complexity: O(1)
        load_factor = size / len(self._arr)
        return load_factor >= HashTableWithCount._LOAD_FACTOR_FOR_RESIZING
    
    def get_hash_index(self, key):
        # compute hash index inside the array bounds
        # Python's hash() is complexity of O(1) average
        return hash(key) % len(self._arr)

    def is_hash_table_too_full(self):
        # check if we have reached the load factor threshold
        return self.check_size(self._size)
        
    def enlarge_hash_table(self):
        # resize the table when it gets too full
        # rehashing all elements has cost O(n)
        
        if self.is_hash_table_too_full():
            # new size ≈ 2*old + 1, then adjusted to next prime
            new_size = len(self._arr) * 2 + 1
            new_size = self.get_next_prime(new_size)
            
            # allocate new table
            new_arr = [None] * new_size
            old_arr = self._arr
            
            # update metadata
            self._arr = new_arr
            self._capacity = new_size
            self._size = 0
            
            # re-insert each active entry into the new table
            for i in range(len(old_arr)):
                if old_arr[i] != None and not old_arr[i].is_removed():
                    key = old_arr[i].get_key()
                    value = old_arr[i].get_value()
                    
                    index = self.get_hash_index(key)
                    
                    # resolve collisions using linear probing
                    while self._arr[index] != None:
                        index = (index + 1) % len(self._arr)
                    self._arr[index] = TableEntry(key, value)
                    self._size += 1
                
    
    def reset_probe_count(self):
        # reset counter before each measurement
        self._probe_count = 0
    
    def get_probes(self):
        # return number of probes performed in last op
        return self._probe_count
    


In [3]:
class LinearProbingWithCount(HashTableWithCount):
    def probe(self, index, key):
        # linear probing: move forward one step
        # Complexity: O(1)
        return (index + 1) % len(self._arr)
        
    
    def locate(self, start_index, key): 
        # search for key by checking sequential slots
        # Time Complexity: O(1) average, O(n) worst-case
        index = start_index
        
        while True:
            slot = self._arr[index]
            
            if slot is None:
                # empty space means key NOT found
                return -1
            if not slot.is_removed() and slot.get_key() == key:
                # key found
                return index
            
            # continue probing
            index = self.probe(index, key)
        
    
    def add(self, key, value):
        # insert or update a key-value pair
        # Time Complexity: O(1) average, O(n) worst-case
        
        # enlarge if load factor exceeded
        if self.is_hash_table_too_full():
            self.enlarge_hash_table()
        
        # starting index from hash
        index = self.get_hash_index(key)
            
        while True:
            if self._arr[index] == None:
                # empty slot: insert here
                self._arr[index] = TableEntry(key, value)
                self._size += 1
                return
            if self._arr[index].is_removed():
                # lazy deletion slot: reuse it
                self._arr[index] = TableEntry(key, value)
                self._size += 1
                return
            if self._arr[index].get_key() == key:
                # key already exists: update value
                self._arr[index].set_value(value)
                return
            
            # collision → probe to next slot
            index = self.probe(index, key)
            self._probe_count += 1
            
    
    def remove(self, key):
        # remove key by marking slot as “removed”
        # Time Complexity: O(1) average, O(n) worst-case
        
        self.reset_probe_count()
        index = self.get_hash_index(key)
        while True:
            if self._arr[index] == None:
                # key not found
                return
            if self._arr[index].get_key() == key:
                # found key → lazy delete
                self._arr[index].set_to_removed()
                self._size -= 1
                return
            else:
                # continue probing
                index = self.probe(index, key)
                self._probe_count += 1
            
    def get_value(self, key):
        # search and return value for a key
        # Time Complexity: O(1) average, O(n) worst-case
        
        self.reset_probe_count()
        index = self.get_hash_index(key)
        while True:
            if self._arr[index] == None:
                return
            if self._arr[index].get_key() == key:
                return self._arr[index].get_value()
            else:
                index = self.probe(index, key)
                self._probe_count += 1
                
    def contains(self, key):
        # check if key exists in the table
        # Time Complexity: O(1) average, O(n) worst-case
        
        self.reset_probe_count()
        index = self.get_hash_index(key)
        while True:
            if self._arr[index] == None:
                return False
            if self._arr[index].get_key() == key:
                return True
            else:
                index = self.probe(index, key)
                self._probe_count += 1
                
    
    def is_empty(self):
        # True if table has no active entries (O(1))
        return self.get_size() == 0
    
    def get_size(self):
        # number of active elements (O(1))
        return self._size
        
    def clear(self):
        # reset table to empty state
        # Time Complexity: O(n)
        for i in range(len(self._arr)):
            self._arr[i] = None
        self._size = 0



In [4]:
# Inherits HashTableWithCount using double hashing
class DoubleHashingWithCount(HashTableWithCount):
    def primary_hash(self, key):
        # first hash function (O(1))
        return hash(key) % len(self._arr)
    
    def secondary_hash(self, key):
        # second hash function used for step size
        # ensures we skip different positions and reduce clustering
        return 1 + (hash(key) % (len(self._arr) - 1))

    def probe(self, index, key):
       # “jump” by a step size determined by the secondary hash function.
       # Complexity: O(1)
        step_size = self.secondary_hash(key)
        
        return (index + step_size) % len(self._arr)
    
    def locate(self, start_index, key):  
        # search using double hashing
        # Time Complexity: O(1) average, O(n) worst-case
        
        index = start_index
        
        while True:
            slot = self._arr[index]
            
            if slot is None:
                return -1
            if not slot.is_removed() and slot.get_key() == key:
                return index
            
            index = self.probe(index, key)
            
    def add(self, key, value):
        # insert or update key using double hashing
        # Time Complexity: O(1) average, O(n) worst-case
        
        if self.is_hash_table_too_full():
            self.enlarge_hash_table()
        
        index = self.get_hash_index(key)
            
        while True:
            if self._arr[index] == None:
                self._arr[index] = TableEntry(key, value)
                self._size += 1
                return
            if self._arr[index].is_removed():
                self._arr[index] = TableEntry(key, value)
                self._size += 1
                return
            if self._arr[index].get_key() == key:
                # update if key exists
                self._arr[index].set_value(value)
                return
            
            # collision → probe using step size
            index = self.probe(index, key)
            self._probe_count += 1
            
    def remove(self, key):
        # remove entry using lazy deletion
        # Time Complexity: O(1) avg, O(n) worst
        self.reset_probe_count()
        index = self.get_hash_index(key)
        while True:
            if self._arr[index] == None:
                return
            if self._arr[index].get_key() == key:
                self._arr[index].set_to_removed()
                self._size -= 1
                return
            else:
                index = self.probe(index, key)
                self._probe_count += 1
                
    def get_value(self, key):
        # return value for key after probing
        # Time Complexity: O(1) avg, O(n) worst
        self.reset_probe_count()
        index = self.get_hash_index(key)
        while True:
            if self._arr[index] == None:
                return
            if self._arr[index].get_key() == key:
                return self._arr[index].get_value()
            else:
                index = self.probe(index, key)
                self._probe_count += 1
                
    def contains(self, key):
        # check if key exists
        # Time Complexity: O(1) avg, O(n) worst
        self.reset_probe_count()
        index = self.get_hash_index(key)
        while True:
            if self._arr[index] == None:
                return False
            if self._arr[index].get_key() == key:
                return True
            else:
                index = self.probe(index, key)
                self._probe_count += 1
                
    
    def is_empty(self):
        # Complexity O(1)
        return self.get_size() == 0

    def get_size(self):
        # Complexity O(1)
        return self._size

    def clear(self):
        # clear entire table (O(n))
        for i in range(len(self._arr)):
            self._arr[i] = None
        self._size = 0

In [5]:
import random
import statistics

class GetStatistics:

    def main(self):
        # Load names
        names_1000 = self.get_1000_names("names_1000.txt")
        names_10000 = self.get_10000_names("names_10000.txt")

        # Create hash tables
        linear_ht = LinearProbingWithCount()
        double_ht = DoubleHashingWithCount()

        print("\n=== Experiment 1: Insert 100 random names ===")
        self.choose_100_names_and_add(linear_ht, double_ht, names_1000)

        print("\n=== Experiment 2: Insert all 1000 names ===")
        linear_ht2 = LinearProbingWithCount()
        double_ht2 = DoubleHashingWithCount()
        self.insert_all_names(linear_ht2, double_ht2, names_1000)

        print("\n=== Experiment 3: Insert all 10,000 names ===")
        linear_ht3 = LinearProbingWithCount()
        double_ht3 = DoubleHashingWithCount()
        self.insert_all_names(linear_ht3, double_ht3, names_10000)
        
    # ----------------------------
    # Compute statistics
    # ----------------------------
    def stats(self, probes_list):
        average = sum(probes_list) / len(probes_list)
        standard_deviation = 0
        if len(probes_list) > 1:
            standard_deviation = statistics.stdev(probes_list)
        return average, standard_deviation

    # ----------------------------
    # Load Names
    # ----------------------------
    def get_1000_names(self, filename):
        with open(filename, "r") as f:
            names = [line.strip() for line in f]
        return names[:1000]    # ensure exactly 1000 names

    def get_10000_names(self, filename):
        with open(filename, "r") as f:
            names = [line.strip() for line in f]
        return names[:10000]   # ensure exactly 10,000 names


    def choose_100_names_and_add(self, linear_ht, double_ht, names):
        selected = random.sample(names, 100)
        linear_probes = []
        double_probes = []

        # Insert into both tables
        for name in selected:
            linear_ht.reset_probe_count()
            double_ht.reset_probe_count()
            
            linear_ht.add(name, 1)
            double_ht.add(name, 1)
            
            linear_probes.append(linear_ht.get_probes())
            double_probes.append(double_ht.get_probes())
            
        # Total probe count
        print("Linear Probing probes:", sum(linear_probes))
        print("Double Hashing probes:", sum(double_probes))
        
        # Statistics
        average_linear, standard_deviation_linear = self.stats(linear_probes)
        average_double, standard_deviation_double = self.stats(double_probes)
        
        print()
        print("Linear Probing Statistics:")
        print(f"  Average probes: {average_linear:.2f}")
        print(f"  Std deviation: {standard_deviation_linear:.2f}")
        
        print()
        print("Double Hashing Statistics:")
        print(f"  Average probes: {average_double:.2f}")
        print(f"  Std deviation: {standard_deviation_double:.2f}")

        
    # ----------------------------
    # Insert all names
    # ----------------------------
    def insert_all_names(self, linear_ht, double_ht, names):
        linear_probes = []
        double_probes = []
        
        # Insert into both tables
        for name in names:
            linear_ht.reset_probe_count()
            double_ht.reset_probe_count()
            
            linear_ht.add(name, 1)
            double_ht.add(name, 1)
            
            linear_probes.append(linear_ht.get_probes())
            double_probes.append(double_ht.get_probes())

        #Total probe count
        print("Linear Probing probes:", sum(linear_probes))
        print("Double Hashing probes:", sum(double_probes))
        
        # Statistics
        average_linear, standard_deviation_linear = self.stats(linear_probes)
        average_double, standard_deviation_double = self.stats(double_probes)
        
        print()
        print("Linear Probing Statistics:")
        print(f"  Average probes: {average_linear:.2f}")
        print(f"  Std deviation: {standard_deviation_linear:.2f}")
        
        print()
        print("Double Hashing Statistics:")
        print(f"  Average probes: {average_double:.2f}")
        print(f"  Std deviation: {standard_deviation_double:.2f}")

    def search_for_100_names(self, linear_ht, double_ht, names):
        selected = random.sample(names, 100)
        linear_probes = []
        double_probes = []

        # Insert into both tables
        for name in selected:
            linear_ht.reset_probe_count()
            double_ht.reset_probe_count()
            
            linear_ht.contains(name)
            double_ht.contains(name)
            
            linear_probes.append(linear_ht.get_probes())
            double_probes.append(double_ht.get_probes())

        #Total probe count
        print("Linear Probing probes:", sum(linear_probes))
        print("Double Hashing probes:", sum(double_probes))
        
        # Statistics
        average_linear, standard_deviation_linear = self.stats(linear_probes)
        average_double, standard_deviation_double = self.stats(double_probes)
        
        print()
        print("Linear Probing Statistics:")
        print(f"  Average probes: {average_linear:.2f}")
        print(f"  Std deviation: {standard_deviation_linear:.2f}")
        
        print()
        print("Double Hashing Statistics:")
        print(f"  Average probes: {average_double:.2f}")
        print(f"  Std deviation: {standard_deviation_double:.2f}")


# Run program
if __name__ == "__main__":
    GetStatistics().main()



=== Experiment 1: Insert 100 random names ===
Linear Probing probes: 5
Double Hashing probes: 5

Linear Probing Statistics:
  Average probes: 0.05
  Std deviation: 0.22

Double Hashing Statistics:
  Average probes: 0.05
  Std deviation: 0.22

=== Experiment 2: Insert all 1000 names ===
Linear Probing probes: 1200
Double Hashing probes: 796

Linear Probing Statistics:
  Average probes: 1.20
  Std deviation: 2.71

Double Hashing Statistics:
  Average probes: 0.80
  Std deviation: 1.43

=== Experiment 3: Insert all 10,000 names ===
Linear Probing probes: 17779
Double Hashing probes: 10928

Linear Probing Statistics:
  Average probes: 1.78
  Std deviation: 3.58

Double Hashing Statistics:
  Average probes: 1.09
  Std deviation: 1.68


# Comment Analysis

#### In my results, double hashing had fewer collisions than linear probing. Linear probing always checks the next spot in the table, so when things start getting full, it creates big clusters of filled spaces. That makes collisions happen more often. Double hashing is better at avoiding that because it “jumps” through the table using a second hash, so it spreads things out more. Because of that, it needs fewer probes and stays faster even when the table gets more full. Linear probing is super simple and works well when the table isn’t very full, but it becomes slow later. Double hashing is a little more complex, but it handles collisions in a much better and more organized way.